In [1]:
from tool import *

In [130]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)

In [135]:
i = 10

In [137]:
sp = spdb.merge(spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :4])

In [143]:
models = sp['Mapping Model.Suffix'].unique()

In [150]:
target_week = sp['Planweek'].unique()[0]

In [251]:
target_week = get_weekname_from('2023-08-07(W32)', 2)
target_week

'2023-08-21(W34)'

In [252]:
get_tcl_boh(models, target_week)

,Mapping Model.Suffix,Rep PMS,TCL BOM,PP,Ship,BOH
0,22MR410-B.AEUQ,22MR410-BB.CEURMV,X3-22M205M3-EUS0AU,840,840.0,0.0
1,22MR410-B.APDQ,22MR410-BB.CPDRMV,X3-22M205M3-TKS0AU,840,0.0,840.0


In [239]:
get_tcl_boh(models, '2023-09-04(W36)')

,Mapping Model.Suffix,Rep PMS,TCL BOM,PP,Ship,BOH
0,22MR410-B.AEUQ,22MR410-BB.CEURMV,X3-22M205M3-EUS0AU,1678,1678.0,0.0
1,22MR410-B.APDQ,22MR410-BB.CPDRMV,X3-22M205M3-TKS0AU,1680,840.0,840.0
2,22MR410-B.ATMQ,22MR410-BB.CTMRMV,X3-22M205M3-THS0AU,4368,4368.0,0.0


In [240]:
models = ['22MR410-B.APDQ', '22MR410-B.AEUQ', '22MR410-B.ATMQ']
# target_week = '2023-10-16(W42)' 

In [242]:
models = ppr['Mapping Model.Suffix'].unique()

# 1. 과거 생산 실적 불러와 DB 변환

In [47]:
filepath = get_filename()

In [48]:
opp = pd.read_excel(filepath, skiprows=1, sheet_name=0)

In [49]:
pp_date = []
for item in opp.columns:
    if type(item) == datetime.datetime:
        pp_date.append(item)

In [50]:
opp = opp[opp['Item']=='PP']

In [51]:
opp = opp[['TCL BOM']+pp_date]

In [52]:
opp = opp.groupby('TCL BOM').sum().convert_dtypes().stack().reset_index()

In [53]:
opp.columns = ['TCL BOM', 'PP Date', 'QTY']

In [54]:
opp = opp[opp['QTY'] > 0]

In [55]:
opp['Rep PMS'] = opp['TCL BOM'].map(tcl_bom_map)

In [56]:
opp['Mapping Model.Suffix'] = opp['Rep PMS'].map(make_dict_from_df(sp[['Rep PMS', 'Mapping Model.Suffix']].drop_duplicates()))

In [58]:
opp['Updated_at'] = datetime.datetime(2023,11,1,21,10)

In [62]:
with open('D:/Data/TCL PP_result.bin', 'wb') as f:
    pickle.dump(ppr, f)

# 2. 생산계획 데이터 가공 후 DB화

In [2]:
filepath = get_filename()

In [3]:
pp = pd.read_excel(filepath, sheet_name=0, skiprows=1)

In [4]:
pp_date = []
for item in pp.columns:
    if type(item) == datetime.datetime:
        pp_date.append(item)

In [5]:
pp = pd.concat([pp.iloc[:, :9], pp[pp_date]], axis=1)

In [6]:
pp.columns = ['Target Week', 'Country', 'To Site', 'ODF', 'TCL BOM', 'Mapping Model.Suffix', 'Rep PMS', 'Production Date', 'WO'] + pp_date

In [7]:
pp = pp.set_index(['Target Week', 'ODF', 'TCL BOM'])[pp_date].stack().reset_index()

In [8]:
pp.columns = ['Target Week', 'ODF', 'TCL BOM', 'PP Date', 'QTY']

In [9]:
pp = pp[pp['QTY'].apply(lambda x:type(x) in [int, float])]

In [10]:
pp['QTY'] = pp['QTY'].astype(int)

In [11]:
pp['Rep PMS'] = pp['TCL BOM'].map(tcl_bom_map)

In [16]:
pp['Updated_at'] = datetime.datetime(2023,11,6,23,24)

In [17]:
pp

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Updated_at
0,10/23W,DOK7516348,X3-24M205M3-AES0AU,2023-10-25,1300,24MR400-BA.CMARMV,2023-11-06 23:24:00
1,10/23W,DOK7516348,X3-24M205M3-AES0AU,2023-10-26,1100,24MR400-BA.CMARMV,2023-11-06 23:24:00
15,10/23W,DOK7516352,X3-24M205M3-COS0AU,2023-10-26,800,24MR400-BA.CWPRMV,2023-11-06 23:24:00
29,10/30W,DOK7516360,X3-22M205M3-HKS0AU,2023-10-26,280,22MR410-BB.CHKRMV,2023-11-06 23:24:00
43,10/23W,DOK7516357,X3-22M205M3-VMS0AU,2023-10-26,440,22MR410-BB.CTVRMV,2023-11-06 23:24:00
...,...,...,...,...,...,...,...
622,12/25W,NaN,X3-22M205M3-THS0AU,2023-12-22,1000,22MR410-BB.CTMRMV,2023-11-06 23:24:00
623,12/25W,NaN,X3-22M205M3-INS0AU,2023-12-22,840,22MR410-BB.CTRRMV,2023-11-06 23:24:00
624,12/25W,NaN,X3-24M205M3-PHS0AU,2023-12-22,500,24MR400-BA.CPHRMV,2023-11-06 23:24:00
625,12/25W,NaN,X3-24M205M3-PHS0AU,2023-12-23,2384,24MR400-BA.CPHRMV,2023-11-06 23:24:00


In [88]:
pp.insert(4, 'Mapping Model.Suffix', pp['Rep PMS'].map(make_dict_from_df(sp[['Rep PMS', 'Mapping Model.Suffix']].drop_duplicates())).fillna('TBD'))

In [89]:
pp

,Target Week,ODF,TCL BOM,Rep PMS,Mapping Model.Suffix,PP Date,QTY,Updated_at
0,10/23W,DOK7516348,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,24MR400-B.AMAQ,2023-10-25,1300,2023-11-06 23:24:00
1,10/23W,DOK7516348,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,24MR400-B.AMAQ,2023-10-26,1100,2023-11-06 23:24:00
15,10/23W,DOK7516352,X3-24M205M3-COS0AU,24MR400-BA.CWPRMV,24MR400-B.AWPQ,2023-10-26,800,2023-11-06 23:24:00
29,10/30W,DOK7516360,X3-22M205M3-HKS0AU,22MR410-BB.CHKRMV,22MR410-B.AHKQ,2023-10-26,280,2023-11-06 23:24:00
43,10/23W,DOK7516357,X3-22M205M3-VMS0AU,22MR410-BB.CTVRMV,22MR410-B.ATVQ,2023-10-26,440,2023-11-06 23:24:00
...,...,...,...,...,...,...,...,...
622,12/25W,NaN,X3-22M205M3-THS0AU,22MR410-BB.CTMRMV,22MR410-B.ATMQ,2023-12-22,1000,2023-11-06 23:24:00
623,12/25W,NaN,X3-22M205M3-INS0AU,22MR410-BB.CTRRMV,22MR410-B.ATRQ,2023-12-22,840,2023-11-06 23:24:00
624,12/25W,NaN,X3-24M205M3-PHS0AU,24MR400-BA.CPHRMV,24MR400-B.APHQ,2023-12-22,500,2023-11-06 23:24:00
625,12/25W,NaN,X3-24M205M3-PHS0AU,24MR400-BA.CPHRMV,24MR400-B.APHQ,2023-12-23,2384,2023-11-06 23:24:00


In [113]:
with open('D:/Data/TCL PP_new.bin', 'rb') as f:
    pp_db = pickle.load(f)

In [114]:
pp_db.shape

(171, 11)

In [115]:
pp_db

,Target Week,Country,ODF,TCL BOM,Mapping Model.Suffix,Rep PMS,Production Date,WO,PP Date,QTY,Updated_at
0,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,NaN,24MR400-BA.CMARMV,NaT,2400,2023-10-19,2200,2023-10-17 00:36:00
1,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,NaN,24MR400-BA.CMARMV,NaT,2400,2023-10-20,200,2023-10-17 00:36:00
2,NaN,NaN,DOK7516364,X3-24M205M3-ARS0AU,NaN,24MR400-BA.CMIRMV,NaT,1440,2023-10-20,1440,2023-10-17 00:36:00
3,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,NaN,24MR400-BA.CMARMV,NaT,2400,2023-10-20,400,2023-10-17 00:36:00
4,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,NaN,24MR400-BA.CMARMV,NaT,2400,2023-10-21,2000,2023-10-17 00:36:00
...,...,...,...,...,...,...,...,...,...,...,...
512,10/30W,Turkey,DOK7516366,X3-27M205M3-TKS0AU,27MR400-B.APDQ,27MR400-BB.CPDRMV,2023-10-26,512,2023-10-27,237,2023-10-25 16:24:00
528,10/30W,Thailand,DOK7516371,X3-22M205M3-THS0AU,22MR410-B.ATMQ,22MR410-BB.CTMRMV,2023-10-30,2084,2023-10-30,1740,2023-10-25 16:24:00
529,10/30W,Thailand,DOK7516371,X3-22M205M3-THS0AU,22MR410-B.ATMQ,22MR410-BB.CTMRMV,2023-10-30,2084,2023-10-31,344,2023-10-25 16:24:00
542,10/30W,Philippines,DOK7516369,X3-22M205M3-PHS0AU,22MR410-B.APHQ,22MR410-BB.CPHRMV,2023-10-31,420,2023-10-31,420,2023-10-25 16:24:00


In [73]:
pp

,Target Week,Country,ODF,TCL BOM,Rep PMS,Production Date,WO,PP Date,QTY,Updated_at
0,10/23W,UAE,DOK7516348,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,2023-10-26,2400,2023-10-25,1300,2023-11-06 23:24:00
1,10/23W,UAE,DOK7516348,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,2023-10-26,2400,2023-10-26,1100,2023-11-06 23:24:00
15,10/23W,Colombia,DOK7516352,X3-24M205M3-COS0AU,24MR400-BA.CWPRMV,2023-10-26,800,2023-10-26,800,2023-11-06 23:24:00
29,10/30W,Hong Kong,DOK7516360,X3-22M205M3-HKS0AU,22MR410-BB.CHKRMV,2023-10-26,280,2023-10-26,280,2023-11-06 23:24:00
43,10/23W,Viet Nam,DOK7516357,X3-22M205M3-VMS0AU,22MR410-BB.CTVRMV,2023-10-26,770,2023-10-26,440,2023-11-06 23:24:00
...,...,...,...,...,...,...,...,...,...,...
622,12/25W,Thailand,NaN,X3-22M205M3-THS0AU,22MR410-BB.CTMRMV,NaT,1764,2023-12-22,1000,2023-11-06 23:24:00
623,12/25W,India,NaN,X3-22M205M3-INS0AU,22MR410-BB.CTRRMV,NaT,840,2023-12-22,840,2023-11-06 23:24:00
624,12/25W,Philippines,NaN,X3-24M205M3-PHS0AU,24MR400-BA.CPHRMV,NaT,2884,2023-12-22,500,2023-11-06 23:24:00
625,12/25W,Philippines,NaN,X3-24M205M3-PHS0AU,24MR400-BA.CPHRMV,NaT,2884,2023-12-23,2384,2023-11-06 23:24:00


In [74]:
pd.concat([pp_db, pp], join='inner')

,Target Week,Country,ODF,TCL BOM,Rep PMS,Production Date,WO,PP Date,QTY,Updated_at
0,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,NaT,2400,2023-10-19,2200,2023-10-17 00:36:00
1,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,NaT,2400,2023-10-20,200,2023-10-17 00:36:00
2,NaN,NaN,DOK7516364,X3-24M205M3-ARS0AU,24MR400-BA.CMIRMV,NaT,1440,2023-10-20,1440,2023-10-17 00:36:00
3,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,NaT,2400,2023-10-20,400,2023-10-17 00:36:00
4,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-BA.CMARMV,NaT,2400,2023-10-21,2000,2023-10-17 00:36:00
...,...,...,...,...,...,...,...,...,...,...
622,12/25W,Thailand,NaN,X3-22M205M3-THS0AU,22MR410-BB.CTMRMV,NaT,1764,2023-12-22,1000,2023-11-06 23:24:00
623,12/25W,India,NaN,X3-22M205M3-INS0AU,22MR410-BB.CTRRMV,NaT,840,2023-12-22,840,2023-11-06 23:24:00
624,12/25W,Philippines,NaN,X3-24M205M3-PHS0AU,24MR400-BA.CPHRMV,NaT,2884,2023-12-22,500,2023-11-06 23:24:00
625,12/25W,Philippines,NaN,X3-24M205M3-PHS0AU,24MR400-BA.CPHRMV,NaT,2884,2023-12-23,2384,2023-11-06 23:24:00


In [95]:
# pp_db = pp_db[pp_db['Updated_at'] != pp_db['Updated_at'].max()] # 최신 Version PP  삭제시 사용

In [96]:
pp_db = pd.concat([pp_db, pp], join='inner').reset_index(drop=True) # pp_db에 신규 pp 추가

In [98]:
# 업데이트된 pp_db를 저장
with open('D:/Data/TCL PP.bin', 'wb') as f:
    pickle.dump(pp_db, f)

In [39]:
with open('D:/Data/TCL PP.bin', 'rb') as f:
    pp_db = pickle.load(f)

In [48]:
pp.head(3)

,Target Week,Country,ODF,TCL BOM,Mapping Model.Suffix,Rep PMS,Production Date,WO,PP Date,QTY,Updated_at
0,10/23W,UAE,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,2023-10-23,2400,2023-10-23,758,2023-10-25 16:24:00
1,10/23W,UAE,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,2023-10-23,2400,2023-10-24,1642,2023-10-25 16:24:00
15,10/23W,UAE,DOK7516347,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,2023-10-24,800,2023-10-24,558,2023-10-25 16:24:00


In [33]:
pp_db = pd.concat([pp_db, pp], join='inner').reset_index(drop=True)
# with open('D:/Data/TCL PP.bin', 'wb') as f:
#     pickle.dump(pp_db, f)

In [6]:
pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].max()]['QTY'].sum()

32799

In [52]:
with open('D:/Data/TCL PP_new.bin', 'wb') as f:
    pickle.dump(pp_db, f)

In [2]:
with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

In [3]:
with open('D:/Data/TCL PP.bin', 'rb') as f:
    pp_db = pickle.load(f)

In [4]:
pp1 = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].unique()[-1]].reset_index(drop=True) # 최신 PP 를 가져옴
pp2 = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].unique()[-2]].reset_index(drop=True)# 이전 Version의 PP를 가져옴
pp_sum = pd.concat([pp1, pp2]).reset_index(drop=True)
pp_sum['Month'] = pp_sum['PP Date'].apply(lambda x:f'{x.isocalendar().year}-{get_month_from_date(x):02d}')
pp_sum['Week Name'] = pp_sum['PP Date'].map(get_weekname)
pp_sum['Series'] = pp_sum['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0])
pp_sum['Day'] = pp_sum['PP Date'].dt.strftime('%a')
pp_sum['Date'] = pp_sum['PP Date'].dt.strftime('%d')
pp_sum = pp_sum.merge(ste[['To Site', 'Region', 'Country']], how='left')

gap_month = pp_sum.groupby(['Region', 'Series', 'Month', 'Updated_at']).sum('QTY').unstack('Updated_at').fillna(0).astype(int)
gap_month['gap'] = gap_month[gap_month.columns[-1]] - gap_month[gap_month.columns[0]]
gap_month = gap_month.unstack('Month').fillna(0).astype(int)
gap_month['Sum'] = gap_month[('gap')].sum(axis=1)
gap_month.loc[('', 'Sum'), :] = gap_month.sum()
gap_month = gap_month.astype(int)

gap_month

QTY                                                gap  \
Updated_at     2023091511                 2023091821                           
Month             2023-08 2023-09 2023-10    2023-08 2023-09 2023-10 2023-08   
Region Series                                                                  
EU     22MR410       1678    1400    1400       1678    1400    1400       0   
       27MR400          0    9856    2176          0    9856    2176       0   
INDIA  22MR410          0       0     840          0       0     840       0   
       24MR400          0       0     800          0       0     800       0   
       27MR400          0       0     512          0       0     512       0   
KOREA  22MR410          0       0     840          0       0     840       0   
       27MR400          0       0    1024          0       0    1024       0   
MEA    22MR410       1680     840    8400       1680     840    8400       0   
       24MR400          0    1600   12000          0    1600   12000       0   
       27MR400          0       0    2560          0    1024    1536       0   
NA     22MR410          0    3024    4368          0    3024    4368       0   
       22MR41A          0       0     840          0       0     840       0   
       27MR400          0   21760    4864          0   18104    8520       0   
       27MR41A          0       0    1280          0       0    1280       0   
SCA    22MR410        840       0    2688          0     840    2688    -840   
       24MR400          0       0    1600          0       0    1600       0   
       27MR400          0       0    1744          0       0    1744       0   
SEA    22MR410       4368    4452    6231       4368    4452    6231       0   
       24MR400          0    4160    6950          0    4160    6950       0   
       27MR400          0       0    9392          0    1280    8112       0   
       Sum           8566   47092   70509       7726   46580   71861    -840   

                               Sum  
Updated_at                          
Month          2023-09 2023-10      
Region Series                       
EU     22MR410       0       0   0  
       27MR400       0       0   0  
INDIA  22MR410       0       0   0  
       24MR400       0       0   0  
       27MR400       0       0   0  
KOREA  22MR410       0       0   0  
       27MR400       0       0   0  
MEA    22MR410       0       0   0  
       24MR400       0       0   0  
       27MR400    1024   -1024   0  
NA     22MR410       0       0   0  
       22MR41A       0       0   0  
       27MR400   -3656    3656   0  
       27MR41A       0       0   0  
SCA    22MR410     840       0   0  
       24MR400       0       0   0  
       27MR400       0       0   0  
SEA    22MR410       0       0   0  
       24MR400       0       0   0  
       27MR400    1280   -1280   0  
       Sum        -512    1352   0